In [6]:
import gc
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm.auto import tqdm
import random
from pathlib import Path
import math

In [7]:
myseed = 1091102  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

In [8]:
def padding(ball_pos , f_range , cond , front = True):
    x = []
    x = ball_pos[cond]

    for _ in range(f_range - len(x)):

        pad = pd.DataFrame([[0,0,0,0,0]],columns= ball_pos.columns)
        if front == True:
            ball_pos = pd.concat([pad , ball_pos])
        else:
            ball_pos = pd.concat([ball_pos , pad])

    return ball_pos

In [9]:
class BadmintonDataset(Dataset):

    def __init__(self,path,tfm,ball_path,files = None):
        super(BadmintonDataset).__init__()
        self.path = path
        
        if files != None:
            self.files = files
        else:
            self.files = [] 
            for i in Path(path).glob('*'):
                self.files.append(i)
        self.ball_pos = pd.read_csv(ball_path)
        self.transform = tfm
        
    
    
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = str(self.files[idx])
        im = Image.open(fname)
        im = self.transform(im)
        
        f_range = 30
        frame = int(fname.split("_")[-2].strip('.'))
        vid = int(fname.split('/')[-1].split("_")[-3].strip('.'))
        label = int(fname.split("_")[-1].strip('.jpg'))
        
        if label == 1: 
            label = 0
        elif label ==2:
            label = 1
        
        cond = (self.ball_pos['VideoName'] == vid) & (abs(self.ball_pos['Frame'] - frame) <= f_range)  
        ball_pos = self.ball_pos[cond]
        
        if len(ball_pos) == 0 or frame < 2:
            ball_pos = pd.concat([pd.DataFrame([[0,frame,0,0,0]],columns= ball_pos.columns) , ball_pos])
            
        prev_cond = ((frame - ball_pos['Frame']) <= f_range) & ((frame - ball_pos['Frame']) > 0)
        ball_pos = padding(ball_pos , f_range , prev_cond)
        follow_cond = ((ball_pos['Frame'] - frame) <= f_range) & ((ball_pos['Frame'] - frame) > 0)
        ball_pos = padding(ball_pos , f_range , follow_cond , front=False)
            
        ball_pos = torch.tensor(ball_pos[['Visibility','X','Y']].values , dtype = torch.float)
        
#         print(fname , len(ball_pos))

        if torch.flatten(ball_pos).size(0) != 183 :
#             print(img.shape ,ball.size(0) , label)
            print(fname)
                 
        return im , torch.flatten(ball_pos) ,label
    
    

In [12]:
class Classifier(nn.Module):
    def __init__(self , nb_classes):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = models.efficientnet_v2_s(weights = models.EfficientNet_V2_S_Weights.IMAGENET1K_V1)
        self.cnn_fc = nn.Linear(1000 , nb_classes)
        
        self.ball_prefc = nn.LazyLinear(100)
        self.ball_attention = nn.TransformerEncoderLayer(100 , 1 , dim_feedforward = 128 , dropout=0.15 , batch_first = True)
        self.ball_fc = nn.LazyLinear(nb_classes)
        self.fin_fc = nn.LazyLinear(nb_classes)
        
    def forward(self, img , ball):
        
        imgout = self.cnn(img)
        imgout = self.cnn_fc(imgout)
        
        ballout = self.ball_prefc(ball)
        ballout = self.ball_attention(ballout)
        ballout = self.ball_fc(ballout)
        out = self.fin_fc(torch.cat((imgout,ballout) , dim = 1))
        
        
        return out

In [17]:
# image_size = (440,310)
image_size = (330,150)
test_tfm = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
])

train_tfm = transforms.Compose([
    transforms.Resize(image_size),
    transforms.RandomAdjustSharpness(1.5, p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomAutocontrast(p=0.5),
    transforms.ColorJitter(0.2,0.2,0.2,0.05),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5,scale=(0.005,0.015),value=(1,1,1)),
])

In [18]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
nb_classes = 2
model = Classifier(nb_classes).to(device)
batch_size = 32
n_epochs = 20
patience = 10
criterion = nn.CrossEntropyLoss(label_smoothing = 0.05)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)
train_valid_ratio = 0.9
_exp_name = "hit_model"

# read_model = "/kaggle/input/models/sample_best.ckpt"

# if read_model != None:
#     model.load_state_dict(torch.load(read_model))

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [22]:
train_set = BadmintonDataset("/kaggle/input/badminton/dataset/dataset/train/images" , tfm=train_tfm , ball_path="/kaggle/input/badminton/csv/all_ball_pos.csv")
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = BadmintonDataset("/kaggle/input/badminton/dataset/dataset/valid/images", tfm=test_tfm , ball_path="/kaggle/input/badminton/csv/all_ball_pos.csv")
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

print(len(train_set))
print(len(valid_set))

img , ball , label = valid_set[0]

print(img.shape ,ball.shape, label)
print(model(img.unsqueeze(0).to(device) , ball.unsqueeze(0).to(device)))

14332
2780
torch.Size([3, 330, 150]) torch.Size([183]) 0
tensor([[ 0.0672, -0.4168]], device='cuda:0', grad_fn=<AddmmBackward0>)


# Train

In [20]:
# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0
train_acc_record = []
train_loss_record = []
valid_acc_record = []
valid_loss_record = []

for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs , ball_pos , labels = batch
        #imgs = imgs.half()
        #print(imgs.shape,labels.shape)

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device) , ball_pos.to(device))
        
        _, preds = torch.max(logits, 1)
        for t, p in zip(labels.view(-1), preds.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1
        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    
    train_acc_record.append(train_acc.to('cpu'))
    train_loss_record.append(train_loss)
    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
    print(confusion_matrix)
    print("All Class Acc")
    print(confusion_matrix.diag()/confusion_matrix.sum(1))

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, ball_pos, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device) , ball_pos.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))
        _, preds = torch.max(logits, 1)
        for t, p in zip(labels.view(-1), preds.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)
    
    valid_acc_record.append(valid_acc.to('cpu'))
    valid_loss_record.append(valid_loss)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
    print(confusion_matrix)
    print("All Class Acc")
    print(confusion_matrix.diag()/confusion_matrix.sum(1))

    # update logs
    if valid_acc > best_acc:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    else:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # save models
    if valid_acc > best_acc:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break

  0%|          | 0/448 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot([*range(1,len(train_acc_record)+1)] , train_acc_record , label = "training")
plt.plot([*range(1,len(train_acc_record)+1)] , valid_acc_record , label = "validation")

plt.xticks(np.arange(0, n_epochs+1, 5))
plt.legend(loc="upper left")

plt.savefig('acc.png')
plt.show()


plt.plot([*range(1,len(train_acc_record)+1)] , train_loss_record , label = "training")
plt.plot([*range(1,len(train_acc_record)+1)] , valid_loss_record , label = "valiidation")

plt.xticks(np.arange(0, n_epochs+1, 5))
plt.legend(loc="upper left")

plt.savefig('loss.png')
plt.show()